In [14]:
import pandas as pd
import numpy as np
from datetime import date, timedelta

# Get list of days in expected format
sdate = date(2020, 1, 22)
today = date.today()
edate = date(today.year, today.month, today.day)
days = [(sdate + timedelta(days=i)).strftime('%m-%d-%Y') for i in range((edate - sdate).days + 1)]

# Merge all daily reports
merged = pd.DataFrame(columns = ['Country', 'State', 'County', 'Date', 'Confirmed', 'Deaths', 'Recovered', 'Confirmed_New', 'Deaths_New', 'Recovered_New'])
error_days = set()
for day in days:
    url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/' + day + '.csv'
    try:
        df = pd.read_csv(url)
        # Fix changes in column names if they exits
        df = df.rename({'Admin2': 'County', 'Province/State':'State', 'Country/Region':'Country', 'Province_State':'State', 'Country_Region':'Country'}, axis=1)
        df.drop([x for x in df.columns.values if x not in merged.columns.values], axis=1, inplace=True)
        df.insert(2, 'Date', [day for i in range(df.shape[0])])
        merged = pd.concat([merged, df])
    except IOError as e:
        print(str(e) + ' for: ' + day)
        error_days.add(day)

# Remove any days that errored out
days = [x for x in days if x not in error_days]

# Standerdize Country Names
pd.options.mode.chained_assignment = None
merged['Country'].replace('United Kingdom', 'UK', inplace=True)
merged['Country'].replace('Mainland China', 'China', inplace=True)
merged['Country'].replace(['Korea, South', 'Republic of Korea'], 'South Korea', inplace=True)
merged['Country'].replace('Iran (Islamic Republic of)', 'Iran',inplace=True)

# Standerdize US State Names
merged['State'] = merged['State'].str.strip()
merged['State'].replace(regex={'^.*Virgin Islands.*$': 'Virgin Islands'}, inplace=True)
merged['State'].replace(regex={'^(.+) \(From Diamond Princess\)$': r'\1'}, inplace=True)
merged['State'].replace(regex={'^.*Princess.*$': 'Cruise Ship'}, inplace=True)
merged['State'].replace(regex={'^.+, (.+)$': r'\1'}, inplace=True)
merged['State'].replace(['District of Columbia', 'D.C.'], 'DC', inplace=True)
merged['State'].replace('Chicago', 'IL', inplace=True)
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
merged['State'].replace(us_state_abbrev, inplace=True)

merged['Confirmed'].fillna(0, inplace=True)
merged['Deaths'].fillna(0, inplace=True)
merged['Recovered'].fillna(0, inplace=True)
merged['State'].fillna('n/a', inplace=True)
merged['County'].fillna('n/a', inplace=True)

# Do this because there are duplicate rows in some datasets
merged = merged.groupby(['Country', 'State', 'County', 'Date'], as_index=False).sum()

# Remove bad data
merged.drop(merged[(merged['State'] == 'French Polynesia') & (merged['Date'] == '03-23-2020')].index, inplace=True)

# Calculate deltas for each date
for country in merged['Country'].unique():
    for state in merged[merged['Country'] == country]['State'].unique():
        for county in merged[(merged['Country'] == country) & (merged['State'] == state)]['County'].unique():
            confirmed = merged[(merged['Country'] == country) & (merged['State'] == state) & (merged['County'] == county)]['Confirmed'].values.tolist()
            confirmed_deltas = [np.nan] + [confirmed[i] - confirmed[i-1] for i in range(1, len(confirmed))]
            merged.loc[(merged['Country'] == country) & (merged['State'] == state) & (merged['County'] == county), 'Confirmed_New'] = confirmed_deltas
            deaths = merged[(merged['Country'] == country) & (merged['State'] == state) & (merged['County'] == county)]['Deaths'].values.tolist()
            deaths_deltas = [np.nan] + [deaths[i] - deaths[i-1] for i in range(1, len(deaths))]
            merged.loc[(merged['Country'] == country) & (merged['State'] == state) & (merged['County'] == county), 'Deaths_New'] = deaths_deltas
            recovered = merged[(merged['Country'] == country) & (merged['State'] == state) & (merged['County'] == county)]['Recovered'].values.tolist()
            recovered_deltas = [np.nan] + [recovered[i] - recovered[i-1] for i in range(1, len(recovered))]
            merged.loc[(merged['Country'] == country) & (merged['State'] == state) & (merged['County'] == county), 'Recovered_New'] = recovered_deltas

merged.to_csv('covid-merged-daily-reports.csv', index=False)

HTTP Error 404: Not Found for: 03-29-2020


In [24]:
# Run verifications - ignore small deviations
df_neg = merged[(merged['County'] != 'Unassigned') & (merged['Confirmed_New'] < -100) | (merged['Deaths_New'] < -50) | (merged['Recovered_New'] < -50)]
if df_neg.shape[0] > 0:
    print('Some deltas are negative!')
    print(df_neg.sort_values('Confirmed_New'))
